# Correlation Power Analysis (Brier et al. 2004)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
device = CaptureDevice.create("CWLITEXMEGA")
device.compile(file=os.path.abspath("../lecture_3/sbox_lookup.c"))
device.flash()

In [ ]:
data = device.capture(
    number_of_traces=500,
    input=lambda _: random.randbytes(16)
)

In [ ]:
# Create a dict with keys 0,...,8 and values containing
# lists of traces where the hamming weight of the first byte equals the key
grouped_data_raw = {
    hw: [d for d in data if lascar.hamming_weight(d["input"][0]) == hw]
    for hw in range(9)
}

In [ ]:
# Calculate the average trace for each group
grouped_data = {
    hw: np.mean(np.array(traces)["trace"], axis=0)
    for hw, traces in grouped_data_raw.items()
}

In [ ]:
# Plot the mean traces
fig = pgo.Figure()
for hw, trace in grouped_data.items():
    fig.add_trace(pgo.Scatter(y=trace, name=f"HW: {hw}"))
fig.show()


In [ ]:
# Plot hw vs trace point
# The trace point is selected where "the traces look different".
fig = pgo.Figure()
for hw, trace in grouped_data.items():
    fig.add_trace(pgo.Scatter(x=[hw], y=[trace[61]]))
fig.show()
# => We see that the hamming weight of the input is proportional
# to the current consumption in the moment where the input is processed.

## 2. Pearson correlation coefficient

An interesting statistical formula to face this problem is given by the *Pearson correlation coefficient*. For two random variables $X, Y$ it is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

In [ ]:
def pearson(xs, ys):
    xmean = np.mean(xs)
    ymean = np.mean(ys)
    return sum((xs - xmean) * (ys - ymean)) / np.sqrt(
        sum((xs - xmean) ** 2) * sum((ys - ymean) ** 2)
    )


fig = pgo.Figure()
size = 50
data1 = 5 * np.array(range(size)) + np.random.uniform(-size / 4, size / 4, size=size)
fig.add_trace(pgo.Scatter(y=data1, name=f"pearson:{pearson(range(size), data1)}"))
data2 = np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10
fig.add_trace(pgo.Scatter(y=data2, name=f"pearson:{pearson(range(size), data2)}"))
data3 = (
    -3 * np.array(range(size)) + np.random.uniform(-size / 4, size / 4, size=size) + 200
)
fig.add_trace(pgo.Scatter(y=data3, name=f"pearson:{pearson(range(size), data3)}"))
data4 = 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(
    -size / 10, size / 10, size=size
)
fig.add_trace(pgo.Scatter(y=data4, name=f"pearson:{pearson(range(size), data4)}"))
fig.show()

## 3. Attack!

In [ ]:
import lascar.tools


def aes_sbox_cpa(traces, key_byte_index=0, trace_point=145):
    # All trace values at the given trace point
    values_at_trace_point = traces["trace"][:, trace_point]
    # Iterate over all possible keys
    pearsons = []
    for guess in range(256):
        # Hamming weights of hypothesis
        hamming_weights = [
            lascar.hamming(lascar.tools.aes.sbox[inp[key_byte_index] ^ guess])
            for inp in traces["input"]
        ]
        # Calculate pearson of these two
        p = pearson(values_at_trace_point, hamming_weights)
        pearsons.append(abs(p))
    print(np.argmax(pearsons))


aes_sbox_cpa(traces=data)

In [ ]:
# Get rid of the trace point as input
def aes_sbox_cpa2(traces, key_byte_index=0):
    # Iterate over all possible keys
    pearsons = []
    for guess in range(256):
        # Iterate over all trace points
        pearsons_per_point = []
        for trace_point in range(traces["trace"].shape[1]):
            # All trace values at the given trace point
            values_at_trace_point = traces["trace"][:, trace_point]
            # Hamming weights of hypothesis
            hamming_weights = [
                lascar.hamming(lascar.tools.aes.sbox[inp[key_byte_index] ^ guess])
                for inp in traces["input"]
            ]
            # Calculate pearson of these two
            p = pearson(values_at_trace_point, hamming_weights)
            pearsons_per_point.append(abs(p))
        # Store the maximum pearson coefficient of all trace points for the current guess
        print(f"Max pearson for guess {guess}: {max(pearsons_per_point):.02f}")
        pearsons.append(max(pearsons_per_point))
    print(np.argmax(pearsons))

aes_sbox_cpa2(traces=data[:20], key_byte_index=7)

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 5</div>

Perform a CPA attack using Lascar's `CpaEngine`.

</div>